In [27]:
import numpy as np
from finlab.data import Data
import math
import datetime
import pandas as pd
data = Data()
close = data.get('adj_close')
def triple_barrier(price, ub, lb, max_period):

    def end_price(s):
        return np.append(s[(s / s[0] > ub) | (s / s[0] < lb)], s[-1])[0]/s[0]
    
    r = np.array(range(max_period))
    
    def end_time(s):
        return np.append(r[(s / s[0] > ub) | (s / s[0] < lb)], max_period-1)[0]

    p = price.rolling(max_period).apply(end_price, raw=True).shift(-max_period+1)
    t = price.rolling(max_period).apply(end_time, raw=True).shift(-max_period+1)
    t = pd.Series([t.index[int(k+i)] if not math.isnan(k+i) else np.datetime64('NaT') 
                   for i, k in enumerate(t)], index=t.index).dropna()

    signal = pd.Series(0, p.index)
    signal.loc[p > ub] = 1
    signal.loc[p < lb] = -1
    ret = pd.DataFrame({'triple_barrier_profit':p, 'triple_barrier_sell_time':t, 'triple_barrier_signal':signal})

    return ret

#ret = triple_barrier(close, 1.07, 0.97, 20)

In [38]:
close = data.get("收盤價")
def br(c,ub=1.07,lb=0.95,max_period=20):
    ret = triple_barrier(close[c],ub,lb,max_period)
    return ret

In [10]:
def get_profit(date,stock):
    try:
        date = datetime.datetime(year=int(date[0:4]), month=int(date[4:6]), day=int(date[6:8]))
        dates = date.strftime("%Y-%m-%d")
        ret = br(stock)
        profit = ret.triple_barrier_profit[date]
        return profit
    except:
        try:
            date=date + datetime.timedelta(days=2)
            dates = date.strftime("%Y-%m-%d")
            ret = br(stock)
            profit = ret.triple_barrier_profit[date]
            return profit
        except:
            pass
        return 1

In [18]:
def br(c):
    close = data.get("收盤價")
    ret = triple_barrier(close[c],1.2,0.95,21)
    return ret